In [1]:
using MatrixNetworks
using Distributions
using Gadfly
using Clustering
using BenchmarkTools, Compat
using DelimitedFiles
using SparseArrays
using LinearAlgebra
using SphericalGraphEmbedding
using Random


┌ Info: Precompiling SphericalGraphEmbedding [abe9faeb-2622-484d-8e33-db466c9fd899]
└ @ Base loading.jl:1342


In [12]:
#dataset = "PolBlogs"
#dataset = "PowerEU"
#dataset = "Facebook"
#dataset = "PowerUS"
#dataset = "HEP"
#dataset = "Internet"
#dataset = "CondMat2003"
#dataset = "EmailEnron"
dataset = "Pokec"


# Initialization
A = 0
r = 0
dim0 = 0

# nb iterations projected power method
n_it_PPM = 30000

# relative objective variation
t = 1e-10

# nb of times vector partition is repeated
n_it_vec_part = 100

# nb updates vec part
n_updates = 100 

# shape of embedding
shape = "Ellipsoidal"

"Ellipsoidal"

In [13]:
# Loading data

if dataset == "PolBlogs"
    A0 = readdlm("data/PolBlogs/PolBlogsAdj.txt")
    lab = readdlm("data/PolBlogs/LabelsPolBlogs.txt")

    dim0 = size(A0)
    A  = zeros(Int64 ,dim0[1],dim0[1])

    for i = 1:dim0[1]
        for j = 1:dim0[1]
            A[i,j] = Int64(A0[i,j])
        end
    end
    A = A+A'
    A = sparse(A)

    A,p = largest_component(A)
    lab = lab[p]
    A0 = 0
    p = 0 
    N = size(A,2)

    # number of columns of initial guess
    r = 20
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "PowerEU"

    list = readdlm("data/PowerEU/ElistPowerEU.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j])
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100
elseif dataset == "Facebook"

    list = readdlm("data/Facebook/facebook-gender_edges.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # here start at 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) # here start at 0

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "PowerUS"

    list = readdlm("data/PowerUS/ElistPowerUS.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j])
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "HEP"

    list = readdlm("data/HEP/hep-year_edges.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) 
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Wikivote"

    list = readdlm("data/Wikivote/wiki-Vote.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) 
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Internet"

    list = readdlm("data/Internet/InternetEList.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "CondMat2003"

    list = readdlm("data/CondMat2003/CondMat2003EList.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100


elseif dataset == "Pokec"

    list = readdlm("data/Pokec/pokec-_edges.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

end
print(dataset)

LoadError: at row 2465673, column 4 : Base.InvalidCharError{Char}('\xe0\x84\xb4'))

In [16]:
list = readdlm("data/Pokec/pokec-_edges.txt")


LoadError: at row 2465673, column 4 : Base.InvalidCharError{Char}('\xe0\x84\xb4'))

In [103]:
community = zeros(Int64,N,1);
x_embed,community =  @time sphere_embed_cluster(A, n_it_PPM, t, n_clusters, n_it_vec_part, n_updates, shape, r)
# beware run twice to get timings without compilation time

 ------- Acc Projected Power Iteration -------
The iteration has become stationary after 158 iterations
 20.392858 seconds (4.76 k allocations: 14.512 GiB, 12.11% gc time)
 -------------- Clustering ------- 
dimension of embedding used for clustering: 50
Number of updates: 29
Number of communities: 12
Modularity: 0.6188964633367515
 -------------------------------------------- 
The first 5 squared singular values divided by N : 
[0.5268717849692212, 0.24816523520327324, 0.15950223862807453, 0.04412958222555406, 0.021320359028276087]
 -------------------------------------------- 
701.894292 seconds (163.90 M allocations: 138.591 GiB, 4.64% gc time)


([-0.004878919571718354 -0.0023620034597827645 … -0.006288780423655884 0.002011909783837253; -0.008385461890389263 0.009689322836740741 … -0.0039502353998368935 -0.005111917250045666; … ; -0.00023786460027159077 0.0018145589255588407 … -0.00046025867317535277 -0.0013134881859114154; -0.0012188821151614684 0.0022149876696621576 … 0.002314117378626288 -0.0004883988138740246], [1, 2, 2, 3, 3, 3, 3, 4, 5, 6  …  6, 3, 3, 2, 4, 3, 3, 3, 4, 4])